In [2]:
#!/usr/bin/env python

import sys
from random import choice
from argparse import ArgumentParser, FileType
from configparser import ConfigParser
from confluent_kafka import Producer

import requests
import numpy as np
import pandas as pd
from datetime import datetime
from datetime import date
from datetime import timedelta
import time
import yfinance as yf

def get_latest_stock_price(stock):
    data = stock.history()
    latest_stock_price = data['Close'].iloc[-1]
    return latest_stock_price

def get_today():
    return date.today()

def get_next_day_str(today):
    return get_date_from_string(today) + timedelta(days=1)

def get_date_from_string(expiration_date):
    return datetime.strptime(expiration_date, "%Y-%m-%d").date()

def get_string_from_date(expiration_date):
    return expiration_date.strftime('%Y-%m-%d')

# read the ticker
ticker = "TSLA"
print('ticker', ticker)

i = 0
# Produce data by repeatedly fetching today's stock prices - feel free to change

# date range
ed = get_today() + timedelta(days=-1)
sd = ed + timedelta(days=-1)
dfvp = yf.download(tickers=ticker, start=sd, end=ed, interval="1m")


ticker TSLA


[*********************100%***********************]  1 of 1 completed


In [3]:
dfvp

Price,Adj Close,Close,High,Low,Open,Volume
Ticker,TSLA,TSLA,TSLA,TSLA,TSLA,TSLA
Datetime,,,,,,
2024-11-12 14:30:00+00:00,339.279999,339.279999,345.839996,339.140015,339.250092,18513521
2024-11-12 14:31:00+00:00,337.825012,337.825012,341.390015,337.480011,339.250092,1265442
2024-11-12 14:32:00+00:00,335.529999,335.529999,338.849915,335.299988,337.777313,1362315
2024-11-12 14:33:00+00:00,338.079987,338.079987,338.980011,335.380096,335.460114,1004810
2024-11-12 14:34:00+00:00,340.429993,340.429993,340.618408,337.149994,338.089996,925410
...,...,...,...,...,...,...
2024-11-12 20:55:00+00:00,328.660004,328.660004,328.999908,328.059998,328.940002,334084
2024-11-12 20:56:00+00:00,328.589996,328.589996,328.850006,327.900085,328.709991,327906


In [10]:
row = dfvp.iloc[0]

In [14]:
dir(row["High"])

['T',
 'TSLA',
 '_AXIS_LEN',
 '_AXIS_ORDERS',
 '_AXIS_TO_AXIS_NUMBER',
 '_HANDLED_TYPES',
 '__abs__',
 '__add__',
 '__and__',
 '__annotations__',
 '__array__',
 '__array_priority__',
 '__array_ufunc__',
 '__bool__',
 '__class__',
 '__column_consortium_standard__',
 '__contains__',
 '__copy__',
 '__deepcopy__',
 '__delattr__',
 '__delitem__',
 '__dict__',
 '__dir__',
 '__divmod__',
 '__doc__',
 '__eq__',
 '__finalize__',
 '__float__',
 '__floordiv__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__iand__',
 '__ifloordiv__',
 '__imod__',
 '__imul__',
 '__init__',
 '__init_subclass__',
 '__int__',
 '__invert__',
 '__ior__',
 '__ipow__',
 '__isub__',
 '__iter__',
 '__itruediv__',
 '__ixor__',
 '__le__',
 '__len__',
 '__lt__',
 '__matmul__',
 '__mod__',
 '__module__',
 '__mul__',
 '__ne__',
 '__neg__',
 '__new__',
 '__nonzero__',
 '__or__',
 '__pandas_priority__',
 '__pos__',
 '__pow__',
 '__radd__',
 

In [18]:
row["High"]

Ticker
TSLA    345.839996
Name: 2024-11-12 14:30:00+00:00, dtype: float64

In [17]:
dfvp.values

array([[3.39279999e+02, 3.39279999e+02, 3.45839996e+02, 3.39140015e+02,
        3.39250092e+02, 1.85135210e+07],
       [3.37825012e+02, 3.37825012e+02, 3.41390015e+02, 3.37480011e+02,
        3.39250092e+02, 1.26544200e+06],
       [3.35529999e+02, 3.35529999e+02, 3.38849915e+02, 3.35299988e+02,
        3.37777313e+02, 1.36231500e+06],
       ...,
       [3.29079987e+02, 3.29079987e+02, 3.29100006e+02, 3.28589996e+02,
        3.28589996e+02, 9.34779000e+05],
       [3.29040009e+02, 3.29040009e+02, 3.29450012e+02, 3.28950012e+02,
        3.29079987e+02, 4.82966000e+05],
       [3.28230011e+02, 3.28230011e+02, 3.29195892e+02, 3.28230011e+02,
        3.29019989e+02, 1.04573600e+06]])

In [3]:
from ta.trend import MACD
import pandas as pd

macd = MACD(pd.DataFrame(data=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10], columns=["dumb"])["dumb"]).macd()